In [38]:
import pandas as pd
import numpy as np
from WindPy import w

In [39]:
w.start()

.ErrorCode=0
.Data=[Already conntected!]

In [3]:
# fname = '\\\\hfm-pubshare\HFM各部门共享\Harvest嘉实全体员工\YANJJ\综合信息查询_成交回报.xls'
fname = '../data/综合信息查询_成交回报.xls'

In [4]:
df = pd.read_excel(fname, index_col=0, converters={'证券代码': str})

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [5]:
df = df[df['委托方向'] == '卖出']

In [6]:
df.loc[df.index, '发生日期'] = pd.to_datetime(df['发生日期'])

In [7]:
def to_windcode(s):
    if s.startswith('6'):
        return s + '.SH'
    else:
        return s + '.SZ'

In [8]:
df['windcode'] = [to_windcode(x) for x in df['证券代码']]

In [9]:
data = w.wss(df['windcode'].drop_duplicates().tolist(), 'ipo_date,mkt,ipo_price')

In [10]:
ipo_date = {k: v for k, v in zip(data.Codes, data.Data[0])}
mkt = {k: v for k, v in zip(data.Codes, data.Data[1])}
ipo_price = {k: v for k, v in zip(data.Codes, data.Data[2])}

In [11]:
df['ipo_date'] = pd.to_datetime([ipo_date[x] for x in df['windcode']])
df['mkt'] = [mkt[x] for x in df['windcode']]
df['ipo_price'] = [ipo_price[x] for x in df['windcode']]

In [12]:
df['days'] = df['发生日期'] - df['ipo_date']

In [13]:
df['days'] = [x.days for x in df['days']]

# 新股判断条件

In [14]:
df = df[df['days'] < 45]

In [55]:
df.head()

,发生日期,基金编号,基金名称,组合名称,证券代码,证券名称,委托方向,指令数量,持仓,最新价,...,今开盘价,是否强平委托,代下达人,是否强平,平仓盈亏,windcode,ipo_date,mkt,ipo_price,days
序号,,,,,,,,,,,,,,,,,,,,,
2,2020-09-03,418.0,嘉实创新成长混合,缺省组合,605123,派克新材,卖出,599,0.0,85.15,...,0.0,非强平,黄文婕,否,0,605123.SH,2020-08-25,主板,30.33,9
3,2020-09-04,418.0,嘉实创新成长混合,缺省组合,688390,N固德威,卖出,2585,0.0,135.00,...,0.0,非强平,闫晶晶,否,0,688390.SH,2020-09-04,科创板,37.93,0
4,2020-09-04,418.0,嘉实创新成长混合,缺省组合,002996,顺博合金,卖出,849,0.0,18.34,...,19.5,非强平,闫晶晶,否,0,002996.SZ,2020-08-28,中小企业板,8.41,7
5,2020-09-07,418.0,嘉实创新成长混合,缺省组合,603155,新亚强,卖出,687,0.0,38.64,...,0.0,非强平,黄文婕,否,0,603155.SH,2020-09-01,主板,31.85,6
6,2020-09-01,418.0,嘉实创新成长混合,缺省组合,603931,格林达,卖出,431,0.0,70.56,...,0.0,非强平,黄文婕,否,0,603931.SH,2020-08-19,主板,21.38,13


# 创新成长打新统计

In [40]:
results = pd.DataFrame(columns=['基金名称', '基金公司名称', 
                                '是否全部卖出（科）', '原持有股数（科）', 
                                '已卖出股数（科）', '卖出持股比例（科）',
                                '科创板股票成本金额', '科创板打新收益',
                                '是否全部卖出（创）', '原持有股数（创）',
                                '已卖出股数（创）', '卖出持股比例（创）',
                                '创业板股票成本金额', '创业板打新收益',
                                '主板打新收益', '打新总收益', '备注'], index=[1])

In [41]:
results.loc[1, '基金名称'] = '嘉实创新成长混合'
results.loc[1, '基金公司名称'] = '嘉实基金管理有限公司'

### 科创板

In [42]:
star = df[df['mkt'] == '科创板']

In [43]:
results.loc[1, '原持有股数（科）'] = (star['成交数量'] + star['持仓']).sum() / 1e4
results.loc[1, '已卖出股数（科）'] = star['成交数量'].sum() / 1e4
results.loc[1, '卖出持股比例（科）'] = results.loc[1, '已卖出股数（科）'] / results.loc[1, '原持有股数（科）']
results.loc[1, '是否全部卖出（科）'] = '是' if results.loc[1, '卖出持股比例（科）'] == 1 else '否'

In [44]:
results.loc[1, '科创板股票成本金额'] = (star['ipo_price']*star['成交数量']).sum()
results.loc[1, '科创板打新收益'] = (star['成交金额']-star['ipo_price']*star['成交数量']).sum()

In [45]:
results

,基金名称,基金公司名称,是否全部卖出（科）,原持有股数（科）,已卖出股数（科）,卖出持股比例（科）,科创板股票成本金额,科创板打新收益,是否全部卖出（创）,原持有股数（创）,已卖出股数（创）,卖出持股比例（创）,创业板股票成本金额,创业板打新收益,主板打新收益,打新总收益,备注
1,嘉实创新成长混合,嘉实基金管理有限公司,是,1.081,1.081,1,570779,509205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 创业板

In [37]:
entr = df[df['mkt'] == '创业板']

In [46]:
results.loc[1, '原持有股数（创）'] = (entr['成交数量'] + entr['持仓']).sum() / 1e4
results.loc[1, '已卖出股数（创）'] = entr['成交数量'].sum() / 1e4
results.loc[1, '卖出持股比例（创）'] = results.loc[1, '已卖出股数（创）'] / results.loc[1, '原持有股数（创）']
results.loc[1, '是否全部卖出（创）'] = '是' if results.loc[1, '卖出持股比例（创）'] == 1 else '否'

In [47]:
results.loc[1, '创业板股票成本金额'] = (entr['ipo_price']*entr['成交数量']).sum()
results.loc[1, '创业板打新收益'] = (entr['成交金额']-entr['ipo_price']*entr['成交数量']).sum()

In [48]:
results

,基金名称,基金公司名称,是否全部卖出（科）,原持有股数（科）,已卖出股数（科）,卖出持股比例（科）,科创板股票成本金额,科创板打新收益,是否全部卖出（创）,原持有股数（创）,已卖出股数（创）,卖出持股比例（创）,创业板股票成本金额,创业板打新收益,主板打新收益,打新总收益,备注
1,嘉实创新成长混合,嘉实基金管理有限公司,是,1.081,1.081,1,570779,509205,否,3.4444,3.2145,0.933254,404760,898858,NaN,NaN,NaN


### 非科创板

In [49]:
main = df[df['mkt'] == '主板']

In [50]:
results.loc[1, '主板打新收益'] = (main['成交金额']-main['ipo_price']*main['成交数量']).sum()

In [51]:
results.loc[1, '打新总收益'] = (df['成交金额'] - df['ipo_price']*df['成交数量']).sum()

In [52]:
results.to_excel('../data/打新统计.xlsx')